In [ ]:
import numpy as np
from keras import layers
import tensorflow as tf
from pathlib import Path
import cv2 as cv

# from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
# from keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# from keras.preprocessing.image import ImageDataGenerator

In [ ]:
input_shape = (256, 256)
data_dir = "data/"

In [ ]:
# Set input shape and number of classes
input_shape = (256, 256)
num_classes = 4

# Create an ImageDataGenerator for data augmentation and normalization
train_datagen = ImageDataGenerator(rescale=1.0 / 255, validation_split=0.2)

# Load images and labels from directories
train_ds = train_datagen.flow_from_directory(
    data_dir,
    target_size=input_shape,
    batch_size=32,
    class_mode="categorical",  # This ensures labels are one-hot encoded
    subset="training",
)

val_ds = train_datagen.flow_from_directory(
    data_dir,
    target_size=input_shape,
    batch_size=32,
    class_mode="categorical",
    subset="validation",
)

In [ ]:
def create_train_save_model(save_path: str, epochs: int = 5) -> None:
    data_dir = "./data/"
    categories = ["Voiga", "Me", "Keanu", "Rest"]

    data_gen = ImageDataGenerator(rescale=1.0 / 255, validation_split=0.2)

    train_data = data_gen.flow_from_directory(
        data_dir,
        target_size=(64, 64),
        batch_size=32,
        class_mode="categorical",
        subset="training",
    )
    val_data = data_gen.flow_from_directory(
        data_dir,
        target_size=(64, 64),
        batch_size=32,
        class_mode="categorical",
        subset="validation",
    )

    model = Sequential(
        [
            Conv2D(32, (3, 3), activation="relu", input_shape=(64, 64, 3)),
            MaxPooling2D((2, 2)),
            Conv2D(64, (3, 3), activation="relu"),
            MaxPooling2D((2, 2)),
            Conv2D(128, (3, 3), activation="relu"),
            MaxPooling2D((2, 2)),
            Flatten(),
            Dense(128, activation="relu"),
            Dense(len(categories), activation="softmax"),
        ]
    )

    model.compile(
        optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
    )

    model.fit(train_data, validation_data=val_data, epochs=epochs)

    model.save(save_path)

In [ ]:
create_train_save_model("./saved_models/model_1_e1.keras", 1)
create_train_save_model("./saved_models/model_1_e5.keras", 5)

In [ ]:
test_model_1 = tf.keras.models.load_model("./saved_models/model_1_e1.keras")
test_model_2 = tf.keras.models.load_model("./saved_models/model_1_e5.keras")

In [ ]:
import cv2
import numpy as np

# Load the trained model
model = tf.keras.models.load_model("./saved_models/model_1_e1.keras")

# Load the face detection model
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

# Start the webcam feed
cap = cv2.VideoCapture(0)
class_names = ["Voiga", "Me", "Keanu", "Rest"]

while True:
    ret, frame = cap.read()
    if not ret:
        break

    img_center_x = frame.shape[1] // 2
    img_center_y = frame.shape[0] // 2

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    for x, y, w, h in faces:
        roi_color = frame[y : y + h, x : x + w]
        roi_color_resized = cv2.resize(
            roi_color, (64, 64)
        )
        roi_color_normalized = (
            roi_color_resized / 255.0
        )
        roi_color_expanded = np.expand_dims(
            roi_color_normalized, axis=0
        )
        predictions = model.predict(roi_color_expanded)
        predicted_class = np.argmax(predictions, axis=1)[0]

        if predicted_class >= len(class_names):
            label = "Rest"
        else:
            label = class_names[predicted_class]

        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        cv2.putText(
            frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2
        )

        face_center_x = x + w // 2
        face_center_y = y + h // 2

        cv2.line(
            frame,
            (img_center_x, img_center_y),
            (face_center_x, face_center_y),
            (0, 255, 0),
            2,
        )

        distance = np.sqrt(
            (face_center_x - img_center_x) ** 2 + (face_center_y - img_center_y) ** 2
        )
        cv2.putText(
            frame,
            f"Dist: {int(distance)}",
            (face_center_x, face_center_y),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (255, 255, 255),
            1,
        )

    cv2.imshow("Face Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
cv2.destroyAllWindows()